In [ ]:
import json
import os

In [ ]:
with open('pneumonia_file_directory.json', 'r') as file:
    data = file.read()
    file_directory = json.loads(data)

with open('pneumonia_data_directory.json', 'r') as file:
    data = file.read()
    data_directory = json.loads(data)

In [ ]:
for cluster, params in file_directory.items():
  for discount, remaining_values in params.items():
    for smoothing, values in remaining_values.items():
      f = file_directory[cluster][discount][smoothing]
      with open(f, 'r') as file:
        data = file.read()
        data_directory[cluster][discount][smoothing] = json.loads(data)
      print('finished for cluster ', cluster, 'discount ', discount, 'smoothing ', smoothing)

finished for cluster  kmeans discount  0.001 smoothing  1e-06
finished for cluster  kmeans discount  0.001 smoothing  0.3
finished for cluster  kmeans discount  0.001 smoothing  1
finished for cluster  kmeans discount  0.01 smoothing  1e-06
finished for cluster  kmeans discount  0.01 smoothing  0.3
finished for cluster  kmeans discount  0.01 smoothing  1
finished for cluster  kmeans discount  0.3 smoothing  1e-06
finished for cluster  kmeans discount  0.3 smoothing  0.3
finished for cluster  kmeans discount  0.3 smoothing  1
finished for cluster  kmeans discount  0.9 smoothing  1e-06
finished for cluster  kmeans discount  0.9 smoothing  0.3
finished for cluster  kmeans discount  0.9 smoothing  1
finished for cluster  minibatch discount  0.001 smoothing  1e-06
finished for cluster  minibatch discount  0.001 smoothing  0.3
finished for cluster  minibatch discount  0.001 smoothing  1
finished for cluster  minibatch discount  0.01 smoothing  1e-06
finished for cluster  minibatch discount  

In [ ]:
import pandas as pd

b_df = pd.read_csv ('b_cluster_summary.csv')
m_df = pd.read_csv ('m_cluster_summary.csv')
k_df = pd.read_csv ('k_cluster_summary.csv')

cluster_summaries = {
    "kmeans": k_df,
    "minibatch": m_df,
    "birch": b_df
}

In [ ]:
!pip install dash
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 22.4 MB/s eta 0:00:00


In [ ]:
from dash import Dash, dcc, html, Input, Output, callback
import plotly.express as px
from scipy.interpolate import griddata
from scipy.ndimage import gaussian_filter
import plotly.graph_objects as go
import numpy as np

import pandas as pd

cluster_names = {0:"Kmeans", 1:"Minibatch Kmeans++", 2:"BIRCH"}
discount_names = {0:"0.001", 1:"0.01", 2:"0.3", 3:"0.9"}
smoothing_names = {0:"1e-06", 1:"0.3", 2:"1"}

maxes = {0: 10, 1: 10, 2: 80}

app1 = Dash(__name__)

app1.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                k_df.columns.unique(),
                'Respiratory Rate',
                id='xaxis-column'
            ),
        ], style={'width': '48%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                k_df.columns.unique(),
                'Heart Rate',
                id='yaxis-column'
            ),
        ], style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
    ]),
    dcc.Graph(id='indicator-graphic'),
    html.H4('Clustering Algorithm'),
    dcc.Slider(0, 2, value=1,
        id='cluster-slider',
               step=1,
        marks={str(key): str(value) for key, value in cluster_names.items()},

    ),
    html.H4('Discount Factor of Max Causal Entropy Algorithm'),
    dcc.Slider(0, 3, value=1,
        id='discount-slider',
               step=1,
        marks={str(key): str(value) for key, value in discount_names.items()},

    ),
    html.H4('Smoothing Value for Creating Transition Dynamics'),
    dcc.Slider(0, 2, value=1,
        id='smoothing-slider',
               step=1,
        marks={str(key): str(value) for key, value in smoothing_names.items()},

    ),
    html.H4('Training Epoch'),
    dcc.Slider(0, 4129, value=4129,
        id='epoch-slider',
               step=1,
        marks={str(key - 1): str(key) for key in [1, 4130]},

    ),

])


@callback(
    Output('indicator-graphic', 'figure'),
    Input('xaxis-column', 'value'),
    Input('yaxis-column', 'value'),
    Input('cluster-slider', 'value'),
    Input('discount-slider', 'value'),
    Input('smoothing-slider', 'value'),
    Input('epoch-slider', 'value'),
    )
def update_graph(xaxis_column_name, yaxis_column_name, cluster, discount, smoothing, epoch):

    cluster_dict = {0: "kmeans", 1: "minibatch", 2: "birch"}

    x = cluster_summaries[cluster_dict[cluster]][xaxis_column_name]
    y = cluster_summaries[cluster_dict[cluster]][yaxis_column_name]
    z = data_directory[cluster_dict[cluster]][discount_names[discount]][smoothing_names[smoothing]][epoch]

    xi = np.linspace(min(x), max(x), 100)
    yi = np.linspace(min(y), max(y), 100)
    xi, yi = np.meshgrid(xi, yi)

    zi = griddata((x, y), z, (xi, yi), method='nearest')

    zi = gaussian_filter(zi, sigma=3)

    zi = np.nan_to_num(zi)

    z_max = maxes[cluster]

    layout = go.Layout(
        scene=dict(
            zaxis=dict(
                range=[0, z_max],  # Set the Z-axis range here,
                title="Reward"
            ),
            xaxis_title=xaxis_column_name,
            yaxis_title=yaxis_column_name

        )
    )

    # Create the 3D scatter plot
    fig = go.Figure(data=[go.Surface(x=xi, y=yi, z=zi, cmax=z_max,
                  hovertemplate =
                  xaxis_column_name+': %{x:.2f}<br>'+
                  yaxis_column_name+': %{y:.2f}<br>'+
                  '<br>Reward: %{z}<br>')],
                    layout=layout)

    # Update plot appearance for a smoother look
    fig.update_traces(contours_z=dict(show=True, usecolormap=True, highlightcolor="limegreen", project_z=True))

    fig.update_traces(
    colorbar=dict(
        title='Reward', # Title of the colorbar
        tickvals=[1, 15, 25], # Positions at which ticks should be displayed
    )
)

    fig.update_yaxes(title='x', visible=True, showticklabels=False)

    return fig


app1.run(debug=True)


<IPython.core.display.Javascript object>

In [ ]:
x = cluster_summaries['kmeans']['Heart Rate']
y = cluster_summaries['kmeans']['Respiratory Rate']
z = data_directory['kmeans']['0.3']['0.3'][-1]

xi = np.linspace(min(x), max(x), 100)
yi = np.linspace(min(y), max(y), 100)
xi, yi = np.meshgrid(xi, yi)

zi = griddata((x, y), z, (xi, yi), method='nearest')

zi = gaussian_filter(zi, sigma=3)

zi = np.nan_to_num(zi)

xaxis_column_name = 'Heart Rate'
yaxis_column_name = 'Respiratory Rate'

z_max = int(max(z)) + 5

layout = go.Layout(
        scene=dict(
            zaxis=dict(
                # range=[0, z_max],  # Set the Z-axis range here,
                title="Reward"
            ),
            xaxis_title=xaxis_column_name,
            yaxis_title=yaxis_column_name

        )
    )

go.Figure(data=[go.Surface(x=xi, y=yi, z=zi,
                  hovertemplate =
                  xaxis_column_name+': %{x:.2f}<br>'+
                  yaxis_column_name+': %{y:.2f}<br>'+
                  '<br>Reward: %{z}<br>')],
                    layout=layout)

